In [1]:
# Libraries

from transformers import AutoModelForCausalLM, AutoTokenizer
import re

In [2]:
# ReAct Agent Loop Components 

# Thought (Reason)
# Action (Tools)
# Observation (Tool Output)

In [3]:
# Initializing LLM
model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [4]:
prompt = "Explain the importance of fast language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [5]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [6]:
class Agent:
    def __init__(self, model, tokenizer, system: str = "") -> None:
            self.model = model
            self.tokenizer = tokenizer
            self.system = system
            self.messages: list = []
            if self.system:
                self.messages.append({"role": "system", "content": system})
    def __call__(self, message):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        messages = self.messages
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response

In [7]:
# System Prompt for the Agent

system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [8]:
def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float:
    planet = planet.lower()
    if planet == "earth":
        return 5.972e24
    if planet == "mars":
        return 6.39e23
    if planet == "jupiter":
        return 1.898e27
    if planet == "saturn":
        return 5.683e26
    if planet == "uranus":
        return 8.681e25
    if planet == "neptune":
        return 1.024e26
    if planet == "mercury":
        return 3.285e23
    if planet == "venus":
        return 4.867e24
    return None

In [9]:
# Query Loop

def loop(max_iterations=10, query: str = ""):

    agent =  Agent(model=model,tokenizer=tokenizer, system=system_prompt)

    tools = ["calculate", "get_planet_mass"]

    next_prompt = query

    i = 0
  
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


In [10]:
import re

# Query Loop with Action Tracking
def loop(max_iterations=10, query: str = ""):
    agent = Agent(model=model, tokenizer=tokenizer, system=system_prompt)
    
    tools = {
        "calculate": calculate,
        "get_planet_mass": get_planet_mass,
    }
    
    next_prompt = query
    i = 0
    completed_actions = set()  # To track already completed actions
    
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)
        
        # Check if the result contains a pause and action command
        if "PAUSE" in result and "Action" in result:
            action_match = re.search(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            if action_match:
                chosen_tool = action_match.group(1)
                arg = action_match.group(2)

                # Check if this action has already been completed to avoid repetition
                if (chosen_tool, arg) in completed_actions:
                    next_prompt = "Observation: Repeated action detected, moving to the next step."
                    print(next_prompt)
                    continue
                
                # Execute the corresponding tool if it exists
                if chosen_tool in tools:
                    result_tool = tools[chosen_tool](arg)
                    next_prompt = f"Observation: {result_tool}"
                    completed_actions.add((chosen_tool, arg))  # Mark action as completed
                else:
                    next_prompt = "Observation: Tool not found"

                print(next_prompt)
                continue
        
        # If the agent provided an answer, end the loop
        if "Answer" in result:
            break

In [11]:
# running the query 

loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")

Thought: First, I need to find the mass of Saturn. Then, I'll add the mass of Earth to the mass of Saturn. Finally, I'll multiply the total by 2.
Action: get_planet_mass: Saturn
PAUSE
Observation: 5.683e+26
Thought: Now that I have the mass of Saturn, I need to add it to the mass of Earth and then multiply the result by 2. I already know the mass of Earth from the previous calculation.
Action: calculate: (5.972e24 + 5.683e26) * 2
PAUSE
Observation: 1.148544e+27
Thought: The calculation is complete. The result is the mass of Earth plus the mass of Saturn, all multiplied by 2.
Answer: The mass of Earth plus the mass of Saturn and all of that times 2 is 1.148544e+27 kg.
